In [ ]:
import numpy as np  # Make sure that numpy is imported
import pandas as pd
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format(
    './model/GoogleNews-vectors-negative300.bin', binary=True)
num_features = 300

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the question and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(questions, model, num_features):
    # Given a set of questions (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    questionFeatureVecs = np.zeros((len(questions),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for question in questions:
        #
        # Print a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d" % (counter, len(questions)))
        # 
        # Call the function (defined above) that makes average feature vectors
        questionFeatureVecs[counter] = makeFeatureVec(question, model, num_features)
        #
        # Increment the counter
        counter = counter + 1
    return questionFeatureVecs

C:\Users\einwo\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def question_to_wordlist( question, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
#     review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-alphanumeric characters
    question_text = re.sub("[^a-zA-Z0-9]"," ", question)
    #
    # 3. Convert words to lower case and split them
    words = question_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [ ]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

first_question = []
for question in train["question1"]:
    first_question.append( question_to_wordlist( str(question) ))

second_question = []
for question in train["question2"]:
    second_question.append( question_to_wordlist( str(question) ))

trainData1 = getAvgFeatureVecs( first_question, model, num_features )

trainData2 = getAvgFeatureVecs( second_question, model, num_features )

print("Creating average feature vecs for test reviews")
test_question_1 = []
for question in test["question1"]:
    test_question_1.append( question_to_wordlist( question ))
test_question_2 = []
for question in test["question2"]:
    test_question_2.append( question_to_wordlist( question ))

testData1 = getAvgFeatureVecs( test_question_1, model, num_features )

testData2 = getAvgFeatureVecs( test_question_2, model, num_features )

The word2vec model converts a word to 300 feature vector. To find the vector a sentence, add all of its word vectors together. Right now the way I find relations between the two vectors is subtracting one from the other, but I'm sure there are better and more novel ways to do this. Also, we can try XGBoost instead of RandomForest. This is what I want you to experiment with. Let me know if you have anything else you'd think would be good to experiment with.

In [ ]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainData2 - trainData1, train["is_duplicate"] )

# Test & extract results 
result = forest.predict( testData2 - testData1 )

# Write the test results 
output = pd.DataFrame( data={"test_id":test["test_id"], "is_duplicate":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )